In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def solveUnderdetermined(Au, bu, xu_0, method='CG', i_max=1e6, epsilon=1e-6):
    
    switcher = {
        'SD': lambda : steepestDescentUnderdetermined(Au, bu, xu_0,
                                                        i_max=i_max, epsilon=epsilon),
        'CG': lambda : conjugateGradientUnderdetermined(Au, bu, xu_0,
                                                        i_max=i_max, epsilon=epsilon),
        'PCG': lambda : conjugateGradientPreconditionedUnderdetermined(Au, bu, xu_0,
                                                        i_max=i_max, epsilon=epsilon),
    }
    x, X, n_steps = switcher.get(method)()

    print('Minimum norm solution =\n', x)
    print('No. of iterations =', n_steps)
    return x

def conjugateGradientUnderdetermined(Au, bu, xu, i_max=1e6, epsilon=1e-6):
    i = 0
    x = Au.T @ xu; b = Au.T @ bu
    r = b - Au.T @ (Au @ x)
    d = r
    delta_new = np.linalg.norm(r) ** 2
    delta_0 = delta_new
    X = []; X.append(x)
    while (i < i_max) & (delta_new > epsilon ** 2 * delta_0):
        q = Au.T @ (Au @ d)
        alpha = delta_new / (d.T.dot(q))
        x = x + alpha * d
        if i % 50 == 0:
            r = b - Au.T @ (Au @ x)
        else:
            r = r - alpha * q
        delta_old = delta_new
        delta_new = np.linalg.norm(r) ** 2
        beta = delta_new / delta_old
        d = r + beta * d
        X.append(x)
        i += 1
    return x, np.array(X), i

def steepestDescentUnderdetermined(Au, bu, xu, i_max=1e6, epsilon=1e-6):
    i = 0
    x = Au.T @ xu; b = Au.T @ bu
    r = b - Au.T @ (Au @ x)
    delta = r.T.dot(r)
    delta_0 = delta
    X = []; X.append(x)
    while (i < i_max) & (delta > epsilon ** 2 * delta_0):
        q = Au.T @ (Au @ r)
        alpha = delta / (r.T.dot(q))
        x = x + alpha * r
        if i % 50 == 0: # for large n: sqrt(n) might be appropriate here instead of 50
            r = b - Au.T @ (Au @ x)
        else:
            r = r - alpha * q
        delta = r.T.dot(r)
        X.append(x);
        i += 1
    return x, np.array(X), i

def conjugateGradientPreconditionedUnderdetermined(Au, bu, xu, i_max=1e6, epsilon=1e-6):
    i = 0
    x = Au.T @ xu; b = Au.T @ bu; A = Au.T @ Au
    r = b - A @ x
    d = precondition(A, r) # M_inverse * r
    delta_new = r.T.dot(d)
    delta_0 = delta_new
    X = []; X.append(x)
    while (i < i_max) & (delta_new > epsilon ** 2 * delta_0):
        q = A @ d
        alpha = delta_new / d.T.dot(q)
        x = x + alpha * d
        if i % 50 == 0:
            r = b - A @ x
        else:
            r = r - alpha * q
        s = precondition(A, r)
        delta_old = delta_new
        delta_new = r.T.dot(s)
        beta = delta_new / delta_old
        d = s + beta * d
        X.append(x)
        i += 1
    return x, np.array(X), i

def precondition(A, r):
    M_inv_r = []
    for i, a_ii in enumerate(A.diagonal()):
        M_inv_r.append(1 / a_ii * r[i])
    return np.array(M_inv_r)

In [3]:
Au = np.array([[2, 3, 4, 1],
               [1, 1, 2, 1],
               [2, 4, 5, 2]])
bu = np.array([[10], [5], [13]])
xu_0 = np.array([[0], [0.5], [0.5]])

In [11]:
x_sol = solveUnderdetermined(Au, bu, xu_0, method='CG')

Minimum norm solution =
 [[0.6]
 [0.8]
 [1.4]
 [0.8]]
No. of iterations = 3


In [5]:
x_ls = np.linalg.lstsq(Au, bu, rcond=None)[0]
print('Least squares solution =\n', x_ls)

Least squares solution =
 [[0.6]
 [0.8]
 [1.4]
 [0.8]]


In [10]:
# Using Moore-Penrose pseudo-inverse
Au_inv = np.linalg.pinv(Au)
x_pinv = Au_inv @ bu
print('Pseudo-inverse solution =\n', x_pinv)

Pseudo-inverse solution =
 [[0.6]
 [0.8]
 [1.4]
 [0.8]]


In [ ]:
# A = np.array([[1, 0.5, 0],
#               [0.5, 1, 0],
#               [0, 0, 2]])
# b = np.array([[2], [12], [5]])
# x_0 = np.array([[.6], [0.7], [0]])